模板版本：V20200422

【项目】项目名称

#### 目录

一、数据获取

二、数据理解

三、数据预览

四、数据处理

五、数据建模

六、模型应用

### 一、数据获取

_温馨提醒：服务器端需提前安装pymysql包。_

#### 连接数据库

In [ ]:
import pymysql.cursors
import pandas as pd
import numpy as np

db = pymysql.connect(host='localhost',user='root',password='密码',db='库名',charset='utf8',cursorclass=pymysql.cursors.DictCursor)
cursor = db.cursor()

print('连接成功！')

 - 备注：

#### 预览数据库中的数据

--// `预处理` //-- < 01 /  >

In [ ]:
sql_1 = '''
SELECT * FROM 表名;
'''
cursor.execute(sql_1)

result = cursor.fetchall()
df_1 = pd.DataFrame(list(result))
#db.close()    # 最后再统一关闭连接

df_1

 - 备注：

### 二、数据理解

#### 字段业务含义

In [ ]:
sql_2 = '''
SELECT
    COLUMN_NAME AS name
    ,COLUMN_COMMENT AS comments
    ,COLUMN_TYPE AS type
FROM information_schema.COLUMNS
WHERE TABLE_NAME='表名'
ORDER BY ORDINAL_POSITION;
'''
cursor.execute(sql_2)

result = cursor.fetchall()
df_type = pd.DataFrame(list(result))

df_type

 - 备注：

#### 字段类型和DataFrame类型对比

In [ ]:
df_type['dataframeDtype'] = pd.DataFrame(df_1.dtypes).reset_index().iloc[:,1:2]
df_type

 - 备注：数据由数据库转成DataFrame后，——（转成DataFrame后，数据类型对应不上的，说明需要进行处理。）

_MySQL及转成DataFrame数据类型对应关系如下：_
 - int -> int64
 - char -> object
 - varchar -> object
 - datetime -> datetime64
 
 异常类型：
 - datetime -> object
 
 以下类型待定：
 - date -> object（不是date？）
 - decimal -> object（不是float？）

### 三、数据预览

_【说明】“数据预览”部分，只做缺失值、异常值、重复值检验，至于判断合理处理方式、具体实现部分，统一放到“数据处理”进行处理。_

#### 缺失值检验：NaN

In [ ]:
df_1.isna().sum()

 - 备注：

#### 缺失值检验：0

In [ ]:
(df_1 == 0).sum()

 - 备注：

#### 缺失值检验：‘’

In [ ]:
(df_1 == '').sum()

 - 备注：

#### 缺失值检验：'0000-00-00 00:00:00'

In [ ]:
(df_1['datetime字段'] == '0000-00-00 00:00:00').any()

 - 备注：

#### 异常值检验：整体预览

// 整体预览：

In [ ]:
df_1.describe(include='all')

 - 备注：

// 数值型变量预览：

In [ ]:
df_1.describe(include=np.number)

 - 备注：

// 非数值型预览（string、categorical）：

In [ ]:
df_1.describe(exclude=np.number)

 - 备注：

#### 异常值检验：字符型变量逐个预览

In [ ]:
df_1['字段1'].unique()

 - 备注：

In [ ]:
df_1['字段2'].unique()

 - 备注：

#### 重复值检验

// 查看整体是否存在重复记录：

In [ ]:
df_1.duplicated().any()

 - 备注：

// 进一步查看重复的记录：

In [ ]:
df_1[df_1.duplicated()]

 - 备注：

### 四、数据处理

### （一）特征工程

_【说明】“特征工程”部分，只做特征的思路准备，具体实现部分，统一放到“生成数据集”进行处理。_

In [ ]:
df_1

#### 特征选择

 - 备注：

#### 特征构造

 - 备注：

#### 特征提取

_【说明】在“生成数据集”部分进行多重共线性检验（相关性检验）及PCA降维处理。_

### （二）预处理

_【说明】“预处理”部分，判断各类缺失值、异常值、重复值的合理处理方式，具体实现部分，统一放到“生成数据集”进行处理。_

#### 缺失值处理

// 字段1缺失值处理：

In [ ]:
df_1['字段1'] = df_1['字段1'].map(lambda x: np.nan if x == '' else x)

 - 备注：

// 字段2缺失值处理：

 - 备注：

#### 异常值处理

 - 备注：

#### 重复值处理

 - 备注：

### （三）生成数据集

_该部分生成df_2_

#### 预处理部分

// 缺失值处理：

// 异常值处理：

// 重复值处理：

#### 特征工程部分

// 特征选择、特征构造：

// 多重共线性检验（相关性检验）：

// PCA降维：

 - 备注：

#### 准备建模数据和待预测数据

In [ ]:
df_model = 

In [ ]:
df_predict = 

### （四）入模处理

_该部分生成df_3_

#### 连续变量分箱

 - 备注：

#### 变量数值化

// 去掉字符型变量（非特征部分）：

 - 备注：

// 生成哑变量：

In [ ]:
df_dummies = pd.get_dummies(df_)
df_dummies

 - 备注：

// 获取变量名称（后续备用）：

In [ ]:
feature_names = df_.drop('y字段', axis=1).columns    # 特征字段名称不含y字段
feature_names

#### 样本均衡

// 样本均衡检验：

In [ ]:
df_['y字段'].value_counts()

In [ ]:
df_['y字段'].value_counts(normalize=True)

// 样本均衡处理：

 - 备注：

#### 标准化数据

--// `预处理` //-- 

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
df_ = scaler.fit_transform(df_)

df_

#### 各数据集名称及范围汇总

- df_1:从数据库中读取的数据集。* rows × * columns


- df_2:完成数据预处理、特征工程之后的数据集。* rows × * columns


- df_model:建模数据集（未进行入模处理）。* rows × * columns


- df_predict:预测数据集（未进行入模处理）。* rows × * columns


- df_3:删除非特征部分的字符型字段。* rows × * columns


- df_4:生成哑变量之后的数据集。* rows × * columns


- df_5_1:保留有评价内容的记录。* rows × * columns
- df_5_2:无评价内容的记录。* rows × * columns


- X:df_5_1的X部分。* rows ×* columns
- y:df_5_1的y部分。* rows × * columns


- X_train:训练集有*个样本
- X_test:测试集有*个


- X_resampled:* rows × * columns
- y_resampled:* rows × * columns


- X_train_res:SMOTE后训练集有*个样本
- X_test_res:SMOTE后测试集有*个样本


- df_predict_result:

### 五、数据建模

#### 建模数据：切分训练集和测试集

// 准备X, y：

--// `预处理` //-- 

In [ ]:
X = 
y = 

// 开始切分：

--// `预处理` //-- 

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

# 显示切分结果
print('训练集有%d个样本.' % X_train.shape[0])
print('测试集有%d个样本.' % X_test.shape[0])

#### 建模调参

// 网格搜索：

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

rfc = RandomForestClassifier(ramdom_state=42, n_jobs=-1)
parameters = {
    'criterion':['gini','entropy'],
    'n_estimators':[10,50,100],
    'min_samples_leaf':[3,4,5,6,7],
    'max_depth':[7,8,9,10]
}
scorings = ['recall','roc_auc']
for scoring in scorings:
    print('===========================================================================')
    print('此次调参的scoring为：%s' % scoring)
    print()
    clf = GridSearchCV(rfc, parameters, scoring=scoring, cv=3)
    clf.fit(X_train, y_train)
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    params = clf.cv_results_['params']
    print('【交叉验证】过程如下：')
    for mean, std, param in zip(means, stds, params):
        print('%s得分为 %0.3f(+-%0.3f)时，参数为：%r' % (scoring, mean, std * 2, param))
    print()
    print('【训练集】最佳参数为：%r' % clf.best_params_)
    print('【训练集】最佳参数得分为：%0.3f' % clf.best_score_)
    print()
    print('【测试集】%s得分为：%0.3f' % (scoring, clf.score(X_test, y_test)))    # 直接获取测试集的得分
    print()
    y_true, y_pred = y_test, clf.predict(X_test)
    reports = classification_report(y_true, y_pred)
    print('【测试集中实际值与预测值对比】模型报告如下：')
    print(reports)

// 最优参数建模：

 - 备注：

#### 模型属性

// 特征重要性：

 - 备注：

#### 模型业务理解

 - 备注：

#### 模型效果评估：指标

 - 备注：

#### 模型效果评估：画图

// 混淆矩阵：

In [ ]:
from sklearn.metrics import plot_confusion_matrix
from matplotlib import pyplot as plt

plot_confusion_matrix(clf, X_test, y_test, cmap=plt.cm.Blues)
plt.show()

// ROC曲线图：

In [ ]:
from sklearn.metrics import plot_roc_curve
from matplotlib import pyplot as plt

plt.rcParams['font.sans-serif'] = ['SimHei']  # 中文字体设置-黑体
plt.rcParams['axes.unicode_minus'] = False  # 解决保存图像是负号'-'显示为方块的问题

ax = plt.gca()
gbc_disp_train = plot_roc_curve(clf, X_train, y_train, ax=ax, name='训练集')
gbc_disp.plot(ax=ax, name='测试集')
plt.show()

 - 备注：

### 六、模型应用

#### 数据风险评估

#### 利用模型进行预测

// 检验待预测数据与建模数据字段是否完全一致（生成哑变量可能出现差异）：

// 模型预测：

// 生成预测结果：

In [ ]:
df_predict_result = 

#### 输出模型结果

// 保存到本地：

// 存入数据库：

#### 结束：关闭数据库连接

In [ ]:
db.close()

print('已关闭数据库连接！')